In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from uuid import uuid4
from datetime import datetime
from pydantic import BaseModel, Field
from typing import List, Optional, Any, Optional, Dict, Literal
from enum import StrEnum

In [8]:
class RoleType(StrEnum):
    """
    Standardized roles for conversation participants.
    
    Defines who is speaking in each message to maintain clear
    dialogue structure and enable role-based processing.
    """
    USER = "user"        # Human user input
    ASSISTANT = "assistant"  # AI assistant response

class Artifact(BaseModel):
    """
    Rich media attachments generated during conversations.
    
    Captures structured outputs like SQL queries, data tables, HTML content,
    or images that are produced as part of the AI's response, enabling
    multimodal conversation history.
    """
    type: Literal["sql", "data", "html", "image"] = Field(..., description="Type of artifact: 'sql' for queries, 'data' for tables/JSON, 'html' for formatted content, 'image' for visual outputs")
    data: Any = Field(..., description="The actual artifact content (query string, data structure, HTML markup, image data)")
    created_at: datetime = Field(default_factory=datetime.now, description="When this artifact was generated")

class Conversation(BaseModel):
    """
    Individual message in a conversation thread.
    
    Represents a single exchange between user and assistant, including
    the message content and any rich artifacts generated. Forms the
    atomic unit of conversational memory.
    """
    conversation_id: str = Field(default_factory=lambda: str(uuid4()), description="Unique message identifier")
    content: str = Field(..., description="The text content of the message")
    role: RoleType = Field(..., description="Who sent this message (user or assistant)")
    created_at: datetime = Field(default_factory=datetime.now, description="Message creation timestamp")
    updated_at: datetime = Field(default_factory=datetime.now, description="Last modification timestamp")
    ai_thought: Any = Field(default=None, description="AI's reasoning process for this message")
    artifacts: Optional[List[Artifact]] = Field(default=None, description="Rich media outputs generated with this message")
    session_id: Optional[str] = Field(default=None, description="Chat session this message belongs to")

Conversation(content="Hi", role=RoleType.USER)

Conversation(conversation_id='9083655f-32a6-4403-aeae-f54a26b24b68', content='Hi', role=<RoleType.USER: 'user'>, created_at=datetime.datetime(2025, 11, 11, 23, 2, 37, 860477), updated_at=datetime.datetime(2025, 11, 11, 23, 2, 37, 860477), ai_thought=None, artifacts=None, session_id=None)

In [9]:
class TraceType(StrEnum):
    """
    Types of provenance traces for episodic memory creation.

    Categorizes the source of information used to create an episode,
    enabling traceability of how summaries were derived from conversations
    or other episodes.
    """
    CONVERSATION = "conversation"
    EPISODE = "episode"

class Trace(BaseModel):
    """
    Provenance tracking for episodic memory creation.
    
    Tracks the source materials used to create an episode, enabling
    full traceability of how summaries were derived from conversations
    or other episodes.
    """
    type: TraceType = Field(..., description="Source type: 'conversation' for message IDs, 'episode' for episode IDs")
    value: Optional[str] = Field(default=None, description="ID of the source conversation or episode")
    created_at: datetime = Field(default_factory=datetime.now, description="When this trace was recorded")

class Episode(BaseModel):
    """
    Episodic memory representing a summarized conversation round.
    
    Episodes capture the essence of interactions, enabling the AI to recall
    past conversations efficiently. They can be derived from multiple conversations
    or synthesized from other episodes for hierarchical memory organization.
    """
    episode_id: str = Field(default_factory=lambda: str(uuid4()), description="Unique episode identifier")
    topic: Optional[List[str]] = Field(default=None, description="Key topics discussed in this episode")
    summary: str = Field(..., description="Concise summary of the conversation round or knowledge synthesis")
    created_at: datetime = Field(default_factory=datetime.now, description="Episode creation timestamp")
    updated_at: datetime = Field(default_factory=datetime.now, description="Last modification timestamp")
    trace: Optional[List[Trace]] = Field(default=None, description="Source conversations/episodes used to create this summary")
    session_id: Optional[str] = Field(default=None, description="Chat session this episode belongs to")
    user_id: Optional[str] = Field(default=None, description="User associated with this episode")
    embedding: Optional[List[float]] = Field(default=None, description="Vector embedding for semantic similarity search")

Episode(summary="This is a summary of the conversation")

Episode(episode_id='c9e71cd8-2486-4485-b998-0ef0c4595349', topic=None, summary='This is a summary of the conversation', created_at=datetime.datetime(2025, 11, 11, 23, 3, 19, 203395), updated_at=datetime.datetime(2025, 11, 11, 23, 3, 19, 203395), trace=None, session_id=None, user_id=None, embedding=None)

In [10]:
class SourceType(StrEnum):
    """
    Types of sources for knowledge retrieval.

    Categorizes the source of information used to create an episode,
    enabling traceability of how summaries were derived from conversations
    or other episodes.
    """
    CONVERSATION = "conversation"
    EPISODE = "episode"
    WEBSITE = "website"
    DOCUMENT = "document"
    API = "api"
    FILE = "file"
    DATABASE = "database"

class RAGContext(BaseModel):
    """
    Retrieval Augmented Generation - Unstructured knowledge sources.
    
    Stores text content from various sources with metadata for semantic
    search and retrieval. Website metadata is included in the metadata field.
    """
    content_id: str = Field(default_factory=lambda: str(uuid4()), description="Unique content identifier")
    content: str = Field(..., description="Raw text content for semantic search")
    source_type: SourceType = Field(..., description="Type of content source")
    source_url: Optional[str] = Field(default=None, description="Original URL or file path")
    embedding: Optional[List[float]] = Field(default=None, description="Vector embedding for similarity search")
    metadata: Optional[Dict[str, Any]] = Field(default=None, description="Source-specific metadata (domain, sitemap, crawl_config, etc.)")
    created_at: datetime = Field(default_factory=datetime.now, description="Content ingestion timestamp")
    updated_at: datetime = Field(default_factory=datetime.now, description="Last update timestamp")

RAGContext(content="This is the content of the document", source_type=SourceType.DOCUMENT)

RAGContext(content_id='bb79ee47-2cb2-499e-801f-384f7e0d3503', content='This is the content of the document', source_type=<SourceType.DOCUMENT: 'document'>, source_url=None, embedding=None, metadata=None, created_at=datetime.datetime(2025, 11, 11, 23, 4, 56, 301282), updated_at=datetime.datetime(2025, 11, 11, 23, 4, 56, 301282))

In [ ]:
class KGContext(BaseModel):
    """
    Knowledge Graph - Structured relationships and entities.
    
    Represents entities and their relationships in a graph structure,
    enabling complex queries about connections between concepts.
    """
    entity_id: str = Field(default_factory=lambda: str(uuid4()), description="Unique entity identifier")
    entity_type: str = Field(..., description="Type of entity (person, place, concept, organization)")
    entity_name: str = Field(..., description="Human-readable entity name")
    relationships: List[Dict[str, Any]] = Field(default_factory=list, description="Connections to other entities")
    properties: Dict[str, Any] = Field(default_factory=dict, description="Entity attributes and properties")
    created_at: datetime = Field(default_factory=datetime.now, description="Entity creation timestamp")

In [ ]:
class Task(BaseModel):
    """Use with procedure"""
    pass